In [1]:
import sys
sys.path.insert(1,'/Users/scarr/MIMS/tensor-hero/Prototyping/onset_detection/Model_Evaluation')
from model_evaluations import *
from model_metric_functions import *
from loss_func import *

In [2]:
import numpy as np

In [52]:
notes_original = np.load('/Users/scarr/MIMS/tensor-hero/example_song/notes_simplified.npy')
notes_original_list = (notes_original[0:400],notes_original[401:800], notes_original[801:1200])
notes_generated = np.load('/Users/scarr/MIMS/tensor-hero/example_song/notes_generated.npy')
notes_generated_list = (notes_generated[0:400],notes_generated[401:800], notes_generated[801:1200])

# notes_generated_list = ('/Users/scarr/MIMS/tensor-hero/example_song/notes_generated.npy', '/Users/scarr/MIMS/tensor-hero/example_song/notes_generated.npy')


In [4]:
#generate single file of generated array
# notes_generated = [item + 2 if index % 90 == 0 else item for index, item in enumerate(notes_original)]
# np.save('/Users/scarr/MIMS/tensor-hero/example_song/notes_generated.npy', notes_generated)

In [17]:
len([i for i  in notes_generated_list[2] if i >5 ])

0

In [5]:
evaluate_model_run(notes_generated_list,notes_original_list)

{'Frequency Table':     % of truth AVG  % of pred AVG
 0         0.954902       0.943216
 1         0.011688       0.011688
 2         0.016702       0.028388
 3         0.007519       0.007519
 4         0.006683       0.006683
 5         0.002506       0.002506
 6         0.000000       0.000000
 7         0.000000       0.000000
 8         0.000000       0.000000
 9         0.000000       0.000000
 10        0.000000       0.000000
 11        0.000000       0.000000
 12        0.000000       0.000000
 13        0.000000       0.000000
 14        0.000000       0.000000
 15        0.000000       0.000000
 16        0.000000       0.000000
 17        0.000000       0.000000
 18        0.000000       0.000000
 19        0.000000       0.000000
 20        0.000000       0.000000
 21        0.000000       0.000000
 22        0.000000       0.000000
 23        0.000000       0.000000
 24        0.000000       0.000000
 25        0.000000       0.000000
 26        0.000000       0.000000
 

In [36]:
# freq_table(notes_original, notes_generated)
temp_freq_table = freq_table(truth=notes_original,output=notes_generated).to_numpy()
temp_freq_table
freq_table_arr = np.zeros((2, 33, 4))

freq_table_arr[0, ...] = temp_freq_table
freq_table_arr[1, ...] = temp_freq_table
freq_table_arr
avg_model_run_freq_table = np.mean(freq_table_arr, axis=0)
# avg_model_run_freq_table = pd.DataFrame(data = avg_model_run_freq_table[:,[1,3]], columns= ['Truth Avg Count', '% of total on avg.','Observed Avg Count','% of total on avg.'])
avg_model_run_freq_table = pd.DataFrame(data = avg_model_run_freq_table[:,[1,3]], columns= ['% of truth AVG','% of pred AVG'])

avg_model_run_freq_table

,% of total on avg.,% of total on avg.
0,0.946476,0.935656
1,0.010027,0.010027
2,0.011974,0.022650
3,0.012479,0.012407
4,0.009089,0.009161
5,0.003823,0.003823
6,0.000793,0.000866
7,0.000289,0.000361
8,0.000361,0.000361
9,0.000433,0.000433


In [22]:
array_metric_avgs = np.matrix((3,4,5,6))
print(array_metric_avgs)
single_metrics_df = pd.DataFrame(data = array_metric_avgs,columns = ['Saturation', 'F1', 'Precision', 'Recall'], index = ['average'])
single_metrics_df


[[3 4 5 6]]


,Saturation,F1,Precision,Recall
average,3,4,5,6


In [3]:
import torch
weights = weights_matrix()

In [4]:
'''MODEL OUTPUT
~~~~~~~~~~~~~~~
output shape: torch.Size([12, 499, 435])
truth notes shape: torch.Size([12, 499])'''



probs = []
for i in range(0, 499):
    prob = np.random.uniform(0,1,435)
    probs.append(prob)
output = torch.tensor(probs)
output = output.unsqueeze(1).repeat(1, 12, 1)
output = torch.transpose(output, 0, 1)
output.shape

torch.Size([12, 499, 435])

In [8]:
#truth array length 4, 6 different chunks
truth = []
for i in range(0, 12):
    truth_notes = np.random.randint(0,435,499)
    truth.append(truth_notes)

truth_tensor = torch.tensor(truth)
truth_tensor.shape


torch.Size([12, 499])

In [9]:
truth_one_hot = torch.nn.functional.one_hot(truth_tensor, num_classes= -1)
#each item in batch has its own one-hot, input is [12,499], transform to [12,499,435]
truth_one_hot.shape

torch.Size([12, 499, 435])

In [10]:

incorrect_vals = truth_one_hot - output
incorrect_vals.shape
# incorrect_values = truth_one_hot - candidates_tensor


torch.Size([12, 499, 435])

In [31]:
truth_tensor.shape

torch.Size([12, 499])

In [33]:
# truth = np.array(truth)
real_weights = weights[truth_tensor,:]
real_weights = torch.tensor(real_weights)

In [20]:
a = np.array(([0,2],[0,1]))
b = np.array(([3,4], [5,6],[7,8], [9,10]))
b[a,:]

array([[[3, 4],
        [7, 8]],

       [[3, 4],
        [5, 6]]])

In [30]:
real_weights.shape
# incorrect_vals.shape

(12, 499, 435)

In [40]:
loss = abs(torch.mul(real_weights,incorrect_vals))
loss

tensor([[[0.8865, 0.0691, 0.6090,  ..., 0.4148, 0.4815, 0.0469],
         [1.1643, 0.1828, 0.7980,  ..., 0.7093, 0.3516, 0.0769],
         [0.0326, 0.4085, 0.1645,  ..., 0.6180, 0.9314, 0.5275],
         ...,
         [0.4342, 0.7627, 0.5314,  ..., 0.2164, 0.4804, 0.5370],
         [0.0143, 0.3558, 0.0131,  ..., 0.0041, 0.7561, 0.5336],
         [0.8831, 0.2247, 0.7481,  ..., 0.7444, 0.4262, 0.5814]],

        [[0.8865, 0.0691, 0.6090,  ..., 0.4148, 0.4815, 0.0469],
         [1.1643, 0.1828, 0.7980,  ..., 0.7093, 0.3516, 0.0769],
         [0.0326, 0.4085, 0.1645,  ..., 0.6180, 0.9314, 0.5275],
         ...,
         [0.4342, 0.7627, 0.5314,  ..., 0.2164, 0.4804, 0.5370],
         [0.0143, 0.3558, 0.0131,  ..., 0.0041, 0.7561, 0.5336],
         [0.7065, 0.1798, 0.5985,  ..., 0.7444, 0.5738, 0.5814]],

        [[0.8865, 0.0691, 0.6090,  ..., 0.4148, 0.4815, 0.0469],
         [1.1643, 0.1828, 0.7980,  ..., 0.7093, 0.3516, 0.0769],
         [0.0326, 0.4085, 0.1645,  ..., 0.6180, 0.9314, 0.

In [42]:
loss_avg = torch.mean(loss)
loss_avg

tensor(0.5338, dtype=torch.float64)